# Manejo de datos faltantes: Imputación

**Imputación**: Estimar los valores faltantes a partir de los datos validos de otras variables y/o otros casos de muestra.

## Problema de trabajar con datos faltantes

- Ignorar los valores faltantes puede introducir sesgos en los sesgos a los análisis y modelos.
- Multiples algoritmos fallaran al no estar preparados (diseñados) para trabajar con `datasets` que tiene valores faltantes.

In [1]:
# Posterior a la versión 22 no es posible importar janitor, por un error en una de las librerias que utiliza pyjanitor
%pip install pyjanitor==0.22.0

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import nhanes.load
import statsmodels.api as sm
import statsmodels.formula.api as smf
import janitor

In [3]:
airquality_df = (
  sm.datasets.get_rdataset("airquality")
  .data
  .clean_names(
    case_type="snake",
  )
  .add_column("year", 1973)
  .assign(
    date = lambda df: pd.to_datetime(df[["year", "month", "day"]])
  )
  .sort_values(by="date")
  .set_index("date")
)
airquality_df

,ozone,solar_r,wind,temp,month,day,year
date,,,,,,,
1973-05-01,41.0,190.0,7.4,67,5,1,1973
1973-05-02,36.0,118.0,8.0,72,5,2,1973
1973-05-03,12.0,149.0,12.6,74,5,3,1973
1973-05-04,18.0,313.0,11.5,62,5,4,1973
1973-05-05,NaN,NaN,14.3,56,5,5,1973
...,...,...,...,...,...,...,...
1973-09-26,30.0,193.0,6.9,70,9,26,1973
1973-09-27,NaN,145.0,13.2,77,9,27,1973
1973-09-28,14.0,191.0,14.3,75,9,28,1973


In [4]:
print(airquality_df.shape)
airquality_df.isna().sum()

(153, 7)


ozone      37
solar_r     7
wind        0
temp        0
month       0
day         0
year        0
dtype: int64

En el dataset `airquality` tenemos 157 observacionesy y 7 variables; de las cuales 2 de ellas tienen valores faltantes.

Si tomamos tal cual el conjunto de datos para ajustar un modelo de regresión líneal y así predecir la temperatura con base en las variables con valores faltantes terminariamos con un modelo sesgado y errono.

**Comprobación:**

In [5]:
(
  smf.ols(
    formula="temp ~ ozone",
    data=airquality_df
  )
  .fit()
  .summary()
  .tables[0]
)

/home/pahoalapizco/anaconda3/envs/exploration_missing_values/lib/python3.9/site-packages/patsy/util.py:672: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return _pandas_is_categorical_dtype(dt)
/home/pahoalapizco/anaconda3/envs/exploration_missing_values/lib/python3.9/site-packages/patsy/util.py:672: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return _pandas_is_categorical_dtype(dt)
/home/pahoalapizco/anaconda3/envs/exploration_missing_values/lib/python3.9/site-packages/patsy/util.py:672: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return _pandas_is_categorical_dtype(dt)


Dep. Variable:,temp,R-squared:,0.488
Model:,OLS,Adj. R-squared:,0.483
Method:,Least Squares,F-statistic:,108.5
Date:,"Mon, 22 Apr 2024",Prob (F-statistic):,2.93e-18
Time:,13:02:02,Log-Likelihood:,-386.27
No. Observations:,116,AIC:,776.5
Df Residuals:,114,BIC:,782.1
Df Model:,1,,
Covariance Type:,nonrobust,,


Al ajustar el modelo utilizando `ozone` como variable independiente para predecir la temperatura, no obtenemos ningun error, warning o algo que nos indique que algo esta mal en el resultado del modelo.

Sin embargo el dataset cuenta con $153$ observaciones, mientras que el modelo solo utilizó $116$, ya que **omitió** $41$ registros que seguraente eran de valores faltantes.

Ahora, ¿qué resultado obtendriamos si ajustamos un modelo introduciendo más variables?

In [6]:
(
  smf.ols(
    formula="temp ~ ozone + solar_r",
    data=airquality_df
  )
  .fit()
  .summary()
  .tables[0]
)

/home/pahoalapizco/anaconda3/envs/exploration_missing_values/lib/python3.9/site-packages/patsy/util.py:672: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return _pandas_is_categorical_dtype(dt)
/home/pahoalapizco/anaconda3/envs/exploration_missing_values/lib/python3.9/site-packages/patsy/util.py:672: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return _pandas_is_categorical_dtype(dt)
/home/pahoalapizco/anaconda3/envs/exploration_missing_values/lib/python3.9/site-packages/patsy/util.py:672: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return _pandas_is_categorical_dtype(dt)
/home/pahoalapizco/anaconda3/envs/exploration_missing_values/lib/python3.9/site-packages/patsy/util.py:672: FutureWarning: is_categor

Dep. Variable:,temp,R-squared:,0.491
Model:,OLS,Adj. R-squared:,0.481
Method:,Least Squares,F-statistic:,52.07
Date:,"Mon, 22 Apr 2024",Prob (F-statistic):,1.47e-16
Time:,13:02:02,Log-Likelihood:,-369.78
No. Observations:,111,AIC:,745.6
Df Residuals:,108,BIC:,753.7
Df Model:,2,,
Covariance Type:,nonrobust,,


Tanto `ozone` como `solar_r` tienen  observaciones con valores faltantes, dandonos como resultado un modelo ajustado solo con $111$ observaciones y no con $153$ (total de registros del dataset).

Si quisieramos comparar el modelo anterior con este nuevo, no podríamos, por que cada uno fue entrenado con un dataset diferente, el primero se entrenó con $116$ observaciones y el segundo con $111$.

Esté es uno de los principales problemas de trabajar con valores faltantes no tratados; a simple vista todo esta bien, pero en realidad tenemos un modelo sesgado o con errores no visibles, y si quiseramos comparar modelos entre si para determinar cual se ajusta mejor, no podríamos porque los datos de entrenamiento son diferentes.

**Más ejemplos:**
- Ajustar modelos de regresión líneal para predecir si el paciente sobrevive o no con base a distintas variables.

In [7]:
survival_df = sm.datasets.get_rdataset("flchain", "survival").data
survival_df.head()

,age,sex,sample.yr,kappa,lambda,flc.grp,creatinine,mgus,futime,death,chapter
0,97,F,1997,5.70,4.860,10,1.7,0,85,1,Circulatory
1,92,F,2000,0.87,0.683,1,0.9,0,1281,1,Neoplasms
2,94,F,1997,4.36,3.850,10,1.4,0,69,1,Circulatory
3,92,F,1996,2.42,2.220,9,1.0,0,115,1,Circulatory
4,93,F,1996,1.32,1.690,6,1.1,0,1039,1,Circulatory


In [8]:
print(survival_df.shape)
survival_df.isna().sum()

(7874, 11)


age              0
sex              0
sample.yr        0
kappa            0
lambda           0
flc.grp          0
creatinine    1350
mgus             0
futime           0
death            0
chapter       5705
dtype: int64

- Variable depndiente: `death`
- Variable predictora: `chapter`

In [9]:
(
  smf.ols(
    formula="death ~ C(chapter)",
    data=survival_df
  )
  .fit()
  .summary()
  .tables[0]
)

/home/pahoalapizco/anaconda3/envs/exploration_missing_values/lib/python3.9/site-packages/patsy/util.py:672: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return _pandas_is_categorical_dtype(dt)
/home/pahoalapizco/anaconda3/envs/exploration_missing_values/lib/python3.9/site-packages/patsy/util.py:672: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return _pandas_is_categorical_dtype(dt)
/home/pahoalapizco/anaconda3/envs/exploration_missing_values/lib/python3.9/site-packages/statsmodels/regression/linear_model.py:1736: RuntimeWarning: divide by zero encountered in scalar divide
  return 1 - self.ssr/self.centered_tss


Dep. Variable:,death,R-squared:,-inf
Model:,OLS,Adj. R-squared:,-inf
Method:,Least Squares,F-statistic:,-143.5
Date:,"Mon, 22 Apr 2024",Prob (F-statistic):,1.00
Time:,13:02:02,Log-Likelihood:,69736.
No. Observations:,2169,AIC:,-1.394e+05
Df Residuals:,2153,BIC:,-1.393e+05
Df Model:,15,,
Covariance Type:,nonrobust,,


- Variable depndiente: `death`
- Variable predictora: `creatinine`

In [10]:
(
  smf.ols(
    formula="death ~ creatinine",
    data=survival_df
  )
  .fit()
  .summary()
  .tables[0]
)

/home/pahoalapizco/anaconda3/envs/exploration_missing_values/lib/python3.9/site-packages/patsy/util.py:672: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return _pandas_is_categorical_dtype(dt)
/home/pahoalapizco/anaconda3/envs/exploration_missing_values/lib/python3.9/site-packages/patsy/util.py:672: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return _pandas_is_categorical_dtype(dt)
/home/pahoalapizco/anaconda3/envs/exploration_missing_values/lib/python3.9/site-packages/patsy/util.py:672: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return _pandas_is_categorical_dtype(dt)


Dep. Variable:,death,R-squared:,0.025
Model:,OLS,Adj. R-squared:,0.025
Method:,Least Squares,F-statistic:,165.0
Date:,"Mon, 22 Apr 2024",Prob (F-statistic):,2.59e-37
Time:,13:02:02,Log-Likelihood:,-4089.4
No. Observations:,6524,AIC:,8183.
Df Residuals:,6522,BIC:,8196.
Df Model:,1,,
Covariance Type:,nonrobust,,


In [11]:
(
  smf.ols(
    formula="death ~ C(chapter) + creatinine",
    data=survival_df
  )
  .fit()
  .summary()
  .tables[0]
)

/home/pahoalapizco/anaconda3/envs/exploration_missing_values/lib/python3.9/site-packages/patsy/util.py:672: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return _pandas_is_categorical_dtype(dt)
/home/pahoalapizco/anaconda3/envs/exploration_missing_values/lib/python3.9/site-packages/patsy/util.py:672: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return _pandas_is_categorical_dtype(dt)
/home/pahoalapizco/anaconda3/envs/exploration_missing_values/lib/python3.9/site-packages/patsy/util.py:672: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return _pandas_is_categorical_dtype(dt)
/home/pahoalapizco/anaconda3/envs/exploration_missing_values/lib/python3.9/site-packages/statsmodels/regression/linear_model.py:1736:

Dep. Variable:,death,R-squared:,-inf
Model:,OLS,Adj. R-squared:,-inf
Method:,Least Squares,F-statistic:,-121.6
Date:,"Mon, 22 Apr 2024",Prob (F-statistic):,1.00
Time:,13:02:03,Log-Likelihood:,63567.
No. Observations:,1962,AIC:,-1.271e+05
Df Residuals:,1945,BIC:,-1.270e+05
Df Model:,16,,
Covariance Type:,nonrobust,,
